<a href="https://colab.research.google.com/github/RudyMartin/esp32-ai-agents/blob/main/camp_navigator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LAB 1 Demo

In [ ]:
# ---------------------------------------------------------------------
# 1️⃣  Install required libraries
# ---------------------------------------------------------------------
!pip install --quiet openai scikit-learn requests

# ---------------------------------------------------------------------
# 2️⃣  Configure API keys
# ---------------------------------------------------------------------
import os, getpass, requests, textwrap, json
from urllib.parse import quote_plus
from openai import OpenAI          # NEW import

# OpenAI key – never commit this to GitHub!
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or getpass.getpass("🔑 Paste your OpenAI key → ")
client = OpenAI(api_key=OPENAI_API_KEY)   # NEW client object

# Optional: GitHub token to dodge rate-limits (60 req/hr unauth)
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN") or ""

# ---------------------------------------------------------------------
# 3️⃣  Recursively download ALL text files from the repo
# ---------------------------------------------------------------------
REPO   = "RudyMartin/dsai-2025"
BRANCH = "main"
API_URL = f"https://api.github.com/repos/{REPO}/git/trees/{BRANCH}?recursive=1"

headers = {"Accept": "application/vnd.github+json"}
if GITHUB_TOKEN:
    headers["Authorization"] = f"token {GITHUB_TOKEN}"

def fetch_tree(url):
    """Return the JSON tree (handles pagination for very large repos)."""
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    data = r.json()
    yield from data.get("tree", [])
    if data.get("truncated"):
        yield from fetch_tree(data["url"])

EXT_OK = (".md", ".txt", ".ino", ".py", ".csv", ".json")
file_paths = [
    item["path"]
    for item in fetch_tree(API_URL)
    if item["type"] == "blob" and item["path"].lower().endswith(EXT_OK)
]

print(f"📂 Found {len(file_paths)} files… downloading")

RAW_BASE = f"https://raw.githubusercontent.com/{REPO}/{BRANCH}/"
docs, loaded = [], 0
for path in file_paths:
    r = requests.get(RAW_BASE + quote_plus(path), headers=headers)
    if r.status_code == 200:
        docs.append(r.text)
        loaded += 1
        if loaded % 25 == 0:
            print(f"  …{loaded} files")
    else:
        print(f"❌ {path} ({r.status_code}) skipped")

full_text = "\n\n".join(docs)
print(f"✅ Loaded {loaded} files • {len(full_text)//1000} KB of text")

# ---------------------------------------------------------------------
# 4️⃣  Chunk text & build TF-IDF index
# ---------------------------------------------------------------------
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

chunks = textwrap.wrap(full_text, width=1000, break_long_words=False)
vectorizer   = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(chunks)

print(f"🔍 Indexed {len(chunks)} chunks")

# ---------------------------------------------------------------------
# 5️⃣  Camp Navigator interactive loop
# ---------------------------------------------------------------------
def navigator_chat(k_top: int = 3, temperature: float = 0.3):
    """
    Simple CLI chatbot.  Type 'quit' to exit.
    """
    print("\n🚀 Camp Navigator ready – ask me anything about the camp (type quit to exit)\n")
    while True:
        query = input("❓ Your question → ").strip()
        if query.lower() in {"quit", "exit"}:
            break

        # SEE & THINK
        q_vec   = vectorizer.transform([query])
        scores  = cosine_similarity(q_vec, tfidf_matrix).flatten()
        top_idxs = scores.argsort()[-k_top:][::-1]
        context  = "\n\n".join(chunks[i] for i in top_idxs)[:4000]   # safety cap

        # DO
        prompt = (
            "You are Camp Navigator, an AI assistant for the Artemis DSAI 2025 camp.\n"
            "Use the camp information below to answer the student clearly and concisely.\n\n"
            f"{context}\n\n"
            f"Q: {query}\nA:"
        )

        resp = client.chat.completions.create(          # NEW call style
            model="gpt-3.5-turbo",
            temperature=temperature,
            messages=[{"role": "user", "content": prompt}],
        )

        # SPEAK
        print("\n🧠 𝙈𝘾𝙋 Trace: See → Think → Do → Speak")
        print("🔍 Top-match snippets shown to GPT (truncated):")
        for idx in top_idxs:
            snippet = chunks[idx][:150].replace("\n", " ")
            print(f"  • …{snippet}…")
        print("\n🤖 Answer:", resp.choices[0].message.content, "\n")

# ---------------------------------------------------------------------
# 6️⃣  Go!
# ---------------------------------------------------------------------
navigator_chat()
